## LGBM Optuna Optimization

## Imports

In [74]:
import optuna
from optuna.integration import LightGBMPruningCallback
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,log_loss
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import lightgbm as lgbm
import numpy as np
from pprint import pprint

from ift6758.models.utils import load_data

from sklearn.ensemble import RandomForestClassifier


## Notebook Variables

In [75]:
RANDOM_SEED = 1237342
# FEATURES = ['shot_type', 'is_rebound', 'distance_to_prev_event', 'is_penalty_shot', 'shot_distance', 'shot_angle',
#             'change_in_angle']
FEATURES = ['shot_distance', 'shot_angle']
target = 'is_goal'

In [71]:
train, test = load_data(features=FEATURES, target=target)
CATEGORICAL_COLUMNS = train.select_dtypes(exclude=["number", "bool_"]).columns

# drop NAs
train = train.dropna()
test = test.dropna()
train = train.reset_index()
test = test.reset_index()

# Convert bools to ints
train[train.select_dtypes([bool]).columns] = train.select_dtypes([bool]).astype(int)
test[test.select_dtypes([bool]).columns] = test.select_dtypes([bool]).astype(int)

# X, y split
X, y = train.drop(columns=[target]), train[target]
X_test, y_test = test.drop(columns=[target]), test[target]

# Scale non-categorical features
scaler = StandardScaler()
X[X.columns.difference(CATEGORICAL_COLUMNS)] = scaler.fit_transform(
    X[X.columns.difference(CATEGORICAL_COLUMNS)])
X_test[X_test.columns.difference(CATEGORICAL_COLUMNS)] = scaler.fit_transform(
    X_test[X_test.columns.difference(CATEGORICAL_COLUMNS)])

# one-hot categorical columns
X = pd.get_dummies(data=X, columns=CATEGORICAL_COLUMNS)
X_test = pd.get_dummies(data=X_test, columns=CATEGORICAL_COLUMNS)

In [76]:
clf = RandomForestClassifier()

clf.fit(X,y)
y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
metrics = {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}
pprint(metrics)
print(np.unique(y_pred,return_counts=True))

{'accuracy': 0.8998812243033348,
 'f1': 0.018627977789718787,
 'precision': 0.4126984126984127,
 'recall': 0.009529045262964999}
(array([0, 1]), array([54599,   126]))


## Optuna Hyper-parameter search

In [65]:
def objective(trial, X, y):
    param_grid = {
        "n_estimators": trial.suggest_int("n_estimators", 50,1000,step=50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 100, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        # "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        # "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        # "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        # "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        # "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1, step=0.1),
        # "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1, step=0.1),
    }

    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)

    cv_scores = np.empty(3)
    for idx, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model = lgbm.LGBMClassifier(objective="binary", **param_grid)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_val, y_val)],
            eval_metric="binary_logloss",
            early_stopping_rounds= 10,
            callbacks=[
                LightGBMPruningCallback(trial, "binary_logloss")
            ],
        )
        preds = model.predict_proba(X_val)
        cv_scores[idx] = log_loss(y_val, preds)

    return np.mean(cv_scores)

In [66]:
study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, X, y)
study.optimize(func, n_trials=20)

[I 2021-11-22 17:47:16,955] A new study created in memory with name: LGBM Classifier


[1]	valid_0's binary_logloss: 0.302625
Training until validation scores don't improve for 10 rounds
[2]	valid_0's binary_logloss: 0.297252
[3]	valid_0's binary_logloss: 0.293702
[4]	valid_0's binary_logloss: 0.291354
[5]	valid_0's binary_logloss: 0.289732
[6]	valid_0's binary_logloss: 0.28859
[7]	valid_0's binary_logloss: 0.287801
[8]	valid_0's binary_logloss: 0.287263
[9]	valid_0's binary_logloss: 0.286876
[10]	valid_0's binary_logloss: 0.286606
[11]	valid_0's binary_logloss: 0.286447
[12]	valid_0's binary_logloss: 0.286311
[13]	valid_0's binary_logloss: 0.286231
[14]	valid_0's binary_logloss: 0.286208
[15]	valid_0's binary_logloss: 0.286159
[16]	valid_0's binary_logloss: 0.286123
[17]	valid_0's binary_logloss: 0.286113
[18]	valid_0's binary_logloss: 0.286123
[19]	valid_0's binary_logloss: 0.286135
[20]	valid_0's binary_logloss: 0.286138
[21]	valid_0's binary_logloss: 0.286151
[22]	valid_0's binary_logloss: 0.28615
[23]	valid_0's binary_logloss: 0.286141
[24]	valid_0's binary_logloss:

/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 4 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_t

[1]	valid_0's binary_logloss: 0.302621
Training until validation scores don't improve for 10 rounds
[2]	valid_0's binary_logloss: 0.297248
[3]	valid_0's binary_logloss: 0.293723
[4]	valid_0's binary_logloss: 0.291387
[5]	valid_0's binary_logloss: 0.289758
[6]	valid_0's binary_logloss: 0.288654


/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 6 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 7 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 8 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 9 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 10 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_

...

/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 13 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 14 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 15 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 16 is already reported.
  warnings.warn(


[14]	valid_0's binary_logloss: 0.285298
[15]	valid_0's binary_logloss: 0.285326
[16]	valid_0's binary_logloss: 0.2853
[17]	valid_0's binary_logloss: 0.285303


/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 17 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 18 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 19 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 20 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 21 is already reported.
  warnings.warn(
/opt/miniconda3/envs/n6758/lib/python3.8/site-packages/optuna/tri

[18]	valid_0's binary_logloss: 0.285302
[19]	valid_0's binary_logloss: 0.285308
[20]	valid_0's binary_logloss: 0.285325
[21]	valid_0's binary_logloss: 0.285357
[22]	valid_0's binary_logloss: 0.285371
[23]	valid_0's binary_logloss: 0.285399
[24]	valid_0's binary_logloss: 0.285405
Early stopping, best iteration is:
[14]	valid_0's binary_logloss: 0.285298


[I 2021-11-22 17:48:22,540] Trial 16 finished with value: 0.28591171040132984 and parameters: {'n_estimators': 500, 'learning_rate': 0.2353215465447941, 'num_leaves': 100, 'max_depth': 5}. Best is trial 3 with value: 0.28583431845482205.
[I 2021-11-22 17:48:22,985] Trial 17 pruned. Trial was pruned at iteration 0.
[I 2021-11-22 17:48:23,498] Trial 18 pruned. Trial was pruned at iteration 0.
[I 2021-11-22 17:48:23,880] Trial 19 pruned. Trial was pruned at iteration 0.


In [67]:
print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")
params = study.best_params
pprint(params)

	Best value (rmse): 0.28583
	Best params:
{'learning_rate': 0.06527777923935238,
 'max_depth': 5,
 'n_estimators': 100,
 'num_leaves': 100}


In [68]:
clf = lgbm.LGBMClassifier(**params)
clf.fit(X,y)
y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
metrics = {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}
pprint(metrics)

{'accuracy': 0.9027385143689624,
 'f1': 0.0007127583749109052,
 'precision': 0.5,
 'recall': 0.0003566333808844508}


In [69]:
np.unique(y_pred,return_counts=True)

(array([0, 1]), array([57655,     4]))